# `S3`: Sensor Lab 3: Portable Forceplate

Force plates (or force platforms) are instruments that measure the ground reaction forces generated by a body standing on or moving across them. They are commonly used to quantify balance, gait and other parameters of biomechanics in medicine and sports.

The simplest force plates measure only the vertical component of the force in the geometric center of the platform. More advanced models measure the three-dimensional components of the single equivalent force applied to the surface and its point of application, usually called the centre of pressure (CoP).

In this portable lab you will be working with a very simple CoP forceplate consisting of four [force sensors](https://www.kiwi-electronics.nl/nl/load-sensor-50kg-2813?language=nl-nl&currency=EUR&gclid=Cj0KCQjwxtSSBhDYARIsAEn0thTvnI1bd6WQD_von2DZ8tOxBhw2jOA-A8ubldtaUDM1Rzt_mVxNFhoaAveIEALw_wcB) coupled to [HX711 amplifiers](https://www.sparkfun.com/products/13879) ([further info on load cells](https://learn.sparkfun.com/tutorials/getting-started-with-load-cells)).


__Outline:__
* [1. Prepare the Raspberry Pi Zero ](#Ch1)
    * [1.1 Connect the forceplate](#Ch11)
    * [1.2 Install required libraries](#Ch12)
* [2. Controlling the forceplate with Python code](#Ch2)
    * [2.1 Calibration procedure](#21)
    * [ 2.2 Reading data from the sensors](#22)
* [3. Creating an Interface to visualize the signals.](#Ch3)
* [4. Force plate measurements](#Ch4)
    * [4.1 What is a Force Plate? ](#Ch41)
    * [4.2 Saving data to file](#Ch42)


## 1. Prepare the Raspberry Pi Zero <a class="anchor" id="Ch1"></a>

### 1.1 Connecting the Forceplate <a class="anchor" id="Ch11"></a>

All ForcePlates have a unique connector scheme. The numbers on the connector are the same but the colour of the wires can be different in your plate compared to the schematic overview. Below the colours are used to indicate the groups belonging to individual loadcells. Finish the table to create an overview of the connections, match the numbers on the connector (female, the one fixed to the lid of the electronics box) with the colours of your wires:

<!-- Figure connector and numbers  -->
<div> <img src="images/ConnectorColours.svg" width="600"> </div>

| Number on connector | Pin Function | Belongs to Loadcell |	Colour of the wire(s) |
| --- | --- | --- | --- |
| 1 | Exitation +   | 1
| 2 | Exitation -   | 1
| 3 | A -           | 1
| 4 | Exitation +   | 2
| 5 | Exitation -   | 2
| 6 | A -           | 2
| 7 | Not connected | NA
| 8 | Not connected | NA
| 9 | Not connected | NA
| 10 | Exitation +   | 3
| 11 | Exitation -   | 3
| 12 | A -           | 3
| 13 | Exitation +   | 4
| 14 | Exitation -   | 4
| 15 | A -           | 4

<!-- Figure breadboard headers  -->
<!-- <div> <img src="images/breadboard_kleur.svg" width="600"> </div> -->

<div> <img src="images/FP_schematic_overview.svg" width="800"> </div>

### 1.2 Install required libraries <a class="anchor" id="Ch12"></a>

To get the board running, we need to install the right libaries and some example code. Therefore, in your terminal, put in the following command:

    >>git clone https://github.com/Morrious/hx711-multi.git
    
More information on this git can be found [here](https://github.com/Morrious/hx711-multi)

In your student folder, there is another __hx711.py__ file. Replace the hx711-file you just downloaded by this Python script. This should solve a bug for you!

## 2. Controlling the forceplate with Python code <a class="anchor" id="Ch2"></a>


### 2.1 Calibration procedure <a class="anchor" id="Ch21"></a>
Each HX711 ADC needs to be calibrated separately in order to account for variance in raw measurements compared to real world weight. For example, the ADC may return a value of 5000 which corresponds to 1 gram. In this case, the weight multiple for this ADC should be set to 5000.

Therefore, run the following calibration sequence with known weights:

1. Open the __/tests/calibrate.py__ in a Python editor on the Raspberry Pi(e.g. Mu).

You will see a prompt asking to:

> Enter SCK/Clock pin:

Your clock pin is connected to __GPIO pin 17__.
Next, you will see:

> Enter DOut/Measurement pin:

These are the input pins in to which your HX711 outputs are connected. See the connection scheme in [Ch1.1](#Ch11) to check the right numbers. 

Repeat the following steps for each of the force sensors individually: 

2. Enter an __output measurement pin__

3. Remove all weights from the plate and press enter.

4. Now place a __known weight__ on the corner of the plate of the output pin you selected in step 1.

5. Enter the weight in kg.

You will now see the __calibration value__. This is the value which will be used to convert the output signal to a measure in kg. 

6. To make the measurement more accurate, repeat step 4 three times. After the third time just press enter without entering a value as known weight

The program will provide you with the average caibration value. __Save these calibration values somewhere.__


### 2.2 Reading data from the sensors <a class="anchor" id="Ch22"></a>

The git that reads multiple HX711 ADC provides some example code to read the sensors.

1. Open the example code _/tests/simple_read_test.py_ on the Raspberry Pi.

2. At the start of the code, change: __sck_pin = 17__, this is again your clock pin.

3. Change the _dout_pins_ to: __dout_pin = [22,27,4,14]__ (see the connection scheme in [Ch1.1](#Ch11)) Note: in the example code there are five outputs, whereas you are only working with four sensors, so four outputs.

4. In _weight_multiples_ enter the four calibration values. Note: enter these values in the same sequence as the _dout_pins_

In [ ]:
# Example code: simple_read_test.py (should come with the git library)

#!/usr/bin/env python3

!!!Not enough imp src path parent: from hx711_multi import HX711
from time import perf_counter
import RPi.GPIO as GPIO  # import GPIO

# init GPIO (should be done outside HX711 module in case you are using other GPIO functionality)
GPIO.setmode(GPIO.BCM)  # set GPIO pin mode to BCM numbering

readings_to_average = 10
sck_pin = 1
dout_pins = [2, 3, 4, 14, 15]
weight_multiples = [-5176, -5500, -5690, -5484, -5455]

# create hx711 instance
hx711 = HX711(dout_pins=dout_pins,
              sck_pin=sck_pin,
              channel_A_gain=128,
              channel_select='A',
              all_or_nothing=False,
              log_level='CRITICAL')
# reset ADC, zero it
hx711.reset()
try:
    hx711.zero(readings_to_average=readings_to_average*3)
except Exception as e:
    print(e)
# uncomment below loop to see raw 2's complement and read integers
# for adc in hx711._adcs:
#     print(adc.raw_reads)  # these are the 2's complemented values read bitwise from the hx711
#     print(adc.reads)  # these are the raw values after being converted to signed integers
hx711.set_weight_multiples(weight_multiples=weight_multiples)

# read until keyboard interrupt
try:
    while True:
        start = perf_counter()

        # perform read operation, returns signed integer values as delta from zero()
        # readings aare filtered for bad data and then averaged
        raw_vals = hx711.read_raw(readings_to_average=readings_to_average)

        # request weights using multiples set previously with set_weight_multiples()
        # This function call will not perform a new measurement, it will just use what was acquired during read_raw()
        weights = hx711.get_weight()

        read_duration = perf_counter() - start
        sample_rate = readings_to_average/read_duration
        print('\nread duration: {:.3f} seconds, rate: {:.1f} Hz'.format(read_duration, sample_rate))
        print(
            'raw',
            ['{:.3f}'.format(x) if x is not None else None for x in raw_vals])
        print(' wt',
              ['{:.3f}'.format(x) if x is not None else None for x in weights])
        # uncomment below loop to see raw 2's complement and read integers
        # for adc in hx711._adcs:
        #     print(adc.raw_reads)  # these are the 2's complemented values read bitwise from the hx711
        #     print(adc.reads)  # these are the raw values after being converted to signed integers
except KeyboardInterrupt:
    print('Keyboard interrupt..')
except Exception as e:
    print(e)

# cleanup GPIO
GPIO.cleanup()

2. Run the code and see that this output is indeed printed. 

__test yourself: Where in the code is this output printed?__

## 3. Creating an Interface to visualize the signals. <a class="anchor" id="Ch3"></a>

Since it is hard to check whether the numbers that are printed make sense, we will create an interface that plots the outputted numbers. Let's create a simple interface with which we can see a running data plot while capturing. [This blog](https://learn.sparkfun.com/tutorials/graph-sensor-data-with-python-and-matplotlib/speeding-up-the-plot-animation) helps you with some inital code. The interface should have the following:
* graph with running data

Optional:
* Try to visualize several lines in one figure of in seperate (sub)plots
* Add a start and stop button





In [ ]:
# Create your own code on the Raspberry Pi


## 4. Forceplate measurements <a class="anchor" id="Ch4"></a>

### 4.1 From force to COP.

In biomechanics, center of pressure (CoP) is the term given to the point of application of the ground reaction force vector. The ground reaction force vector represents the sum of all forces acting between a physical object and its supporting surface. Analysis of the center of pressure is common in studies on human postural control and gait. Changes in motor control are reflected in changes in the center of pressure.

A forceplate can be used to track the CoP during standing balance. Commercial forceplates (e.g. [Kistler](https://www.kistler.com/en/product/type-9286b/)) have four three-dimensional force sensors in the four corners of the plate. These forceplates are quit expensive (around >10k). In your setup the force sensors are one-dimensional sensors, measuring forces in a single direction only: perpedicular to the forceplate. 

1. Estimate the CoP from the output of the force sensors. Use the following diagram for estimation. 

<div> <img src="images/Kistler_FP_CoP.png" width="600"> </div>
<div> <img src="images/Kistler_FP_CoP_tables.png" width="600"> </div>

2. Implement the CoP estimation in your code. 

__Check yourself: What are the assumptions you had to do to estimate the COP with these sensors?__


In [ ]:
## Write your own code on the RP to extract the CoP from the measurements



### 4.2 Save IMU Measurements to a Data File

In the previous sections you have extracted and plotted live force plate data using the libraries. However, we cannot analyse data if this isn't saved somewhere.

To do this, you are going to need to know how to generate unique timestamped filenames ([X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb)) and how to write to CSV files ([X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb)). You should then produce a python script, `logging_FP_data.py`, that logs (writes) your force plate data to a file.

> 🏆 **Challenge**: Go through the [X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb) and [X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb) "eXtra Content" materials.
>
> - After going through [X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb), you should know how to write CSV files
> - After going through [X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb), you should know how to generate timestamped file names
> - Save a new Python3 file called `logging_FP_data.py`
> - Write code in `logging_FP_data.py` that creates a datafile `data/output_yourdatestring.csv` (e.g. `data/output_20220404-114200.csv`)
> - Remember to close your data file at the end of the acquisition
>
> 💡 **Tips**:
>
> - Don't forget to import the `datetime` library at the start of your script
> - Think about how you would like to export the data. What will be your headers? CSV files typically have a header line that describes each column.
> - The rows of your data file will be filled with the IMU data you are measuring

In [ ]:
# 💡 tip: you may need to build a row of your CSV cell-by-cell

from datetime import datetime

row = []
row.append(datetime.now())  # append a timestamp in the first column
row.append(value1)
row.append(value2)

# (and then write the `row` data array to your data file via a `csv.writer`)